# Cape Basin Super Resolution

A simple notebook to test assorted SR models operating on the Cape Basin dataset

## Preparation

In [1]:
%matplotlib ipympl
import torch
import xarray as xa
import hydra, os
from fmod.base.util.config import fmconfig, cfg, ConfigContext
from fmod.model.sres.manager import ResultsAccumulator
from fmod.base.io.loader import TSet
from fmod.base.util.ops import fmbdir, fmtp
import matplotlib.pyplot as plt

In [2]:
hydra.initialize(version_base=None, config_path="../config")
device = ConfigContext.set_device()
def pkey( model: str, tset: TSet, ltype: str ): return '-'.join([model,tset.value,ltype])

task="sres"
model = 'dbpn' # [ 'dbpn', 'edsr', 'srdn', 'unet', 'vdsr', 'mscnn' ]
dataset="LLC4320-v1"
scenario="s4.1"
fmconfig( task, model, dataset, scenario )
fsize = 6.0

### Results data
Read results summary from previous runs:

In [6]:
accum_results = ResultsAccumulator(task,dataset,scenario)
results = accum_results.read( fmbdir('processed') )

plots_data = {}
for tset in [ TSet.Validation, TSet.Test ]:
    for ltype in [ 'model_loss', 'interp_loss' ]:
        plot_data = plots_data.setdefault( pkey(model,tset,ltype), {} )
        for result in results:
            if result['model'].strip() == model and result['tset'].strip() == tset.value:
                plot_data[ result['epoch'] ] = result[ltype]
                
x, y = {}, {}
for tset in [ TSet.Validation, TSet.Test ]:
    pdata = plots_data[ pkey(model,tset,'model_loss') ]
    x[ tset ] = list(pdata.keys())
    y[ tset ] = list(pdata.values())
                
fmt = { TSet.Validation: "b", TSet.Test: "r" }

In [ ]:
with plt.ioff():
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=[fsize*2,fsize], layout="tight")

for tset in [ TSet.Validation.value, TSet.Test.value ]:    
    axs.plot( x[ tset ], y[ tset ], fmt[tset], label=tset )
    
plt.show()